title: What levers can we pull to have more eligible Installment Loans users?
author: Helder Silva 
date: 2022-03-03
region: EU
tags: installment loans, bank products, eligibility, transactions, core markets
summary: Lowering the eligible transaction from 50€ limit all the way down to 10€ would result in having 331.7K users with eligible transactions (corresponding to 61% of all eligible MAUs, a 21.1% increase from our current limit). On the other hand, increasing the upper transaction limit from 500€ seems to have almost no effect in increasing the number of users with eligible transactions, we only see a 0.7pp increase when increasing the upper limit all the way to 1000€. As for blacklisted MCCs, we can see that if we were to not exclude ATM transactions from the product moving forward, we would add about 140.7K users to the group of users with eligible transactions (25.9% of all eligible MAUs). When looking into user eligibility for the Installments Loans product in other core markets, we can see that France is the market with the most eligible users (389K users), which corresponds to about 60% of the number of eligible users we saw for Germany.

<div class="alert alert-block alert-success">
    <H1>What levers can we pull to have more eligible Installment Loans users?</H1>
</div>

To answer this question, we will look into users who were eligible for this product in the end of January 2022, and check their transaction activity since the public launch of this product (July 1st 2021). 

We will be looking into:
- [How many users are currently eligible for this product in Germany?](#section1)
- [How can we increase the eligibility for card transactions?](#section2)
 - [Decreasing Lower Limits](#section2.1)
 - [Increasing Upper Limits](#section2.2)
 - [Reconsidering Blacklisted MCCs](#section2.3)
- [What does eligibility look like for other core markets outside of Germany?](#section3)
 - [All Eligible Users - All Legal Entities](#section3.1)
 - [All Eligible Users - EU Legal Entity](#section3.2)
 - [Percentage of eligible users - All Legal Entities](#section3.3)
 - [Percentage of eligible users - EU Legal Entities](#section3.4)
 - [Card Transactions Eligibility](#section3.5)
- [Can we increase eligibility by adding other transaction types?](#section4)

The main insights we found here were:
- Lowering the eligible transaction from 50€ limit all the way down to 10€ would result in having 331.7K users with eligible transactions (corresponding to 61% of all eligible MAUs, a 21.1% increase from our current limit).
- On the other hand, increasing the upper transaction limit from 500€ seems to have almost no effect in increasing the number of users with eligible transactions, we only see a 0.7pp increase when increasing the upper limit all the way to 1000€.
- As for blacklisted MCCs, we can see that if we were to not exclude ATM transactions from the product moving forward, we would add about 140.7K users to the group of users with eligible transactions (25.9% of all eligible MAUs).
- When looking into user eligibility for the Installments Loans product in other core markets, we can see that France is the market with the most eligible users (389K users), which corresponds to about 60% of the number of eligible users we saw for Germany.

Future Recommendations:
- Even though we could technically lower transaction limits for this product below 50€, we should also look into whether users would be interested in taking loans with these lower limits. From a data perspective, we can see [here](https://metabase-product.tech26.de/question/7505-distribution-of-loans-per-loan-bucket-and-number-of-loans?start_date=2021-07-01&end_date=2022-01-31) that the distribution of loans between July 2021 and January 2022 tends to have a bigger percentage of the loans closer to the lower limit (50€), but it could also be interesting to interview some users to understand if/ for which purposes would they take loans with lower limits.


In [1]:
%%capture
cd/app/

In [2]:
%%capture
!pip install duckdb
!pip install altair

In [3]:
import pandas as pd
from utils.datalib_database import df_from_sql
import utils.altair_functions as af
import duckdb

con = duckdb.connect(database=":memory:", read_only=False)

In [4]:
deu_elig_card_txns_df = df_from_sql(
    "redshiftreader", "select * from dev_dbt.temp_tbil_elig_txns_20220131_deu"
)

In [5]:
oth_elig_card_txns_df = df_from_sql(
    "redshiftreader", "select * from dev_dbt.temp_tbil_elig_txns_20220131_oth"
)

In [6]:
elig_users_df = df_from_sql(
    "redshiftreader",
    "research/product/bank_products/20220209_increase_installment_loans_eligiblity/current_elig_users.sql",
)

In [8]:
all_elig_txns_df = df_from_sql(
    "redshiftreader",
    "research/product/bank_products/20220209_increase_installment_loans_eligiblity/all_eligible_tnxs.sql",
)

In [9]:
grouped_elig_users_query = """
select 
'KYCc DEU users (K)' as label,
kycc_users::numeric/1000 as value 
from elig_users_df
where trim(tnc_country_group) = 'DEU'
union all 
select 
'DEU MAUs (K)' as label,
n_maus::numeric/1000 as value 
from elig_users_df
where trim(tnc_country_group) = 'DEU'
union all 
select 
'Lisbon Eligible Users (K)' as label,
n_lisbon_eligible_users::numeric/1000 as value 
from elig_users_df
where trim(tnc_country_group) = 'DEU'
union all 
select 
'Installments Eligible Users (K)' as label,
n_eligible_users::numeric/1000 as value 
from elig_users_df
where trim(tnc_country_group) = 'DEU'
union all 
select 
'Installments Eligible MAUs (K)' as label,
n_eligible_maus::numeric/1000 as value 
from elig_users_df
where trim(tnc_country_group) = 'DEU'
"""
con.register("elig_users_df", elig_users_df)
grouped_elig_users_df = con.execute(grouped_elig_users_query).fetchdf()

<a id='section1'></a>
# How many users are currently eligible for this product in Germany?

For this exercise, we only look into users that are eligible for the Installments Loans product as of January 31st, 2022. The eligibility criteria for this product at this point in time is as follows:
- Has an eligible Lisbon score (<= 12). For this we look into our credit score service Einsteinium and check which users have an eligible score for this product.
- Has German T&Cs
- Doesn’t have an on-going dunning process
- Haven’t had a write-off


Also, since eligible users should have eligible transactions for this product (and therefore be MAUs), we will look into these two levels of eligibility. Below we can see that as of our reference date, 991.3K KYCc users we have in Germany as of our reference date, and that about 72% of those are MAUs.

On top of that, we split the Lisbon eligibility from the dunning/ write-off one, and it seems that we only have about 4.2K users being excluded due to these extra eligibility steps.

In [10]:
af.column_single_label(grouped_elig_users_df, af.teal, "label", "value", 800, 400, "-y")

alt.LayerChart(...)

In [11]:
grouped_elig_users_perc_query = """
select 
'% eligible users out of kycc DEU users' as label,
perc_eligible_users_out_of_kycc_users as value 
from elig_users_df
where trim(tnc_country_group) = 'DEU'
union all 
select 
'% eligible MAUs out of DEU MAUs' as label,
perc_eligible_maus_out_of_maus as value 
from elig_users_df
where trim(tnc_country_group) = 'DEU'
"""
grouped_elig_users_perc_df = con.execute(grouped_elig_users_perc_query).fetchdf()

Here we can see that when adding the granularity of being an MAU, we reach 76% of eligible MAUs out of all MAUs in Germany. In the next steps we will move on to understand what percentage of these users/ MAUs actually have eligible transactions.

In [12]:
af.bar_single_label(
    grouped_elig_users_perc_df, af.petrol, "value", "label", 700, 200, "-y"
)

alt.LayerChart(...)

In [13]:
week_elig_txns_query = """
select 
'Eligible Users' as label,
txn_date as txn_week,
round(n_txn_eligible_users::numeric/geu.n_eligible_users, 3)*100 as perc_users
from deu_elig_card_txns_df ect
inner join elig_users_df geu using (tnc_country_group)
where not blacklisted_mcc
and txn_date > '2021-07-01' and txn_date < '2022-01-31'
and timeframe = 'week'
and trim(tnc_country_group) = 'DEU'
union all
select 
'Eligible MAUs' as label,
txn_date as txn_week,
round(n_txn_eligible_users::numeric/geu.n_eligible_maus, 3)*100 as perc_users
from deu_elig_card_txns_df ect
inner join elig_users_df geu using (tnc_country_group)
where not blacklisted_mcc
and txn_date > '2021-07-01' and txn_date < '2022-01-31'
and timeframe = 'week'
and trim(tnc_country_group) = 'DEU'
order by 1, 2
"""
con.register("deu_elig_card_txns_df", deu_elig_card_txns_df)
week_elig_txns_df = con.execute(week_elig_txns_query).fetchdf()

Below we can see weekly percentages of eligible users/ MAUs with eligible transactions. We can see that these are quite stable over time, with the percentage of eligible MAUs being consistently higher than the eligible users (which is expected since in this case MAUs are consistently a subset of all users).

In [14]:
af.line_multi(week_elig_txns_df, "label", "txn_week", "perc_users", 800, 400, "x")

alt.Chart(...)

In [15]:
month_elig_txns_query = """
select 
'Eligible Users' as label,
txn_date as txn_month,
round(n_txn_eligible_users::numeric/geu.n_eligible_users, 3)*100 as perc_users
from deu_elig_card_txns_df ect
inner join elig_users_df geu using (tnc_country_group)
where not blacklisted_mcc
and timeframe = 'month'
and trim(tnc_country_group) = 'DEU'
union all
select 
'Eligible MAUs' as label,
txn_date as txn_month,
round(n_txn_eligible_users::numeric/geu.n_eligible_maus, 3)*100 as perc_users
from deu_elig_card_txns_df ect
inner join elig_users_df geu using (tnc_country_group)
where not blacklisted_mcc
and timeframe = 'month'
and trim(tnc_country_group) = 'DEU'
order by 1, 2
"""
month_elig_txns_df = con.execute(month_elig_txns_query).fetchdf()

We can see a similar pattern for the monthly eligible transactions, with the main difference being the consistently higher percentages of users (which is also to be expected since extending the time frame from a week to a month increases the chances for a user to transact within that time frame).

In [16]:
af.line_multi(month_elig_txns_df, "label", "txn_month", "perc_users", 800, 400, "x")

alt.Chart(...)

Below we can find all percentages of weekly/ monthly users with eligible transactions divided by all users in Germany and by eligible users only. In order to reduce complexity moving forward, we will be focusing on monthly MAUs and the division by eligible users only (since users who have eligible transactions are also MAUs and users who are not eligible are not relevant for this product independently of their transaction activity). Therefore, our baseline of eligible users with eligible transactions is 40%, and moving forward we will look into how we can increase this ratio.

In [17]:
elig_txns_query = """
with totals as (
select 
timeframe || ' txn eligible MAUs/ '||timeframe || 'ly DEU users' as label,
txn_date,
round(n_txn_eligible_users::numeric/geu.n_eligible_users, 3)*100 as eligible_users,
round(n_txn_eligible_users::numeric/geu.kycc_users, 3)*100 as all_users
from deu_elig_card_txns_df ect
inner join elig_users_df geu
using (tnc_country_group)
where not blacklisted_mcc
and trim(tnc_country_group) = 'DEU'
union all
select 
timeframe || ' txn eligible MAUs/ '||timeframe || 'ly DEU MAUs' as label,
txn_date,
round(n_txn_eligible_users::numeric/geu.n_eligible_maus, 3)*100,
round(n_txn_eligible_users::numeric/geu.n_maus, 3)*100 
from deu_elig_card_txns_df ect
inner join elig_users_df geu
using (tnc_country_group)
where not blacklisted_mcc
and trim(tnc_country_group) = 'DEU'
)
select label, 
round(avg(all_users), 1) as "Divided by all DEU users/ MAUs",
round(avg(eligible_users), 1) as "Divided by eligible users only"
from totals
group by 1
order by 1
"""
elig_txns_df = con.execute(elig_txns_query).fetchdf()
elig_txns_df

,label,Divided by all DEU users/ MAUs,Divided by eligible users only
0,month txn eligible MAUs/ monthly DEU MAUs,30.6,40.0
1,month txn eligible MAUs/ monthly DEU users,21.9,33.6
2,week txn eligible MAUs/ weekly DEU MAUs,11.8,15.4
3,week txn eligible MAUs/ weekly DEU users,8.4,12.9


<a id='section2'></a>
# How can we increase the eligibility for card transactions?

<a id='section2.1'></a>
## Decreasing Lower Limits

First, we will look into how many more users we could add to the user base if the lower transaction limit for this product would be as low as 10€ instead of the current 50€. We can see that lowering the limit all the way down to this value would result in having 331.7K users with eligible transactions (corresponding to 61% of all eligible MAUs, a 21.1% increase from our current limit).

In [18]:
lower_limit_query = """
with unions as (
select 
'eligible users' as label, 
txn_date,
n_txn_eligible_users as n_users
from deu_elig_card_txns_df ect
where timeframe = 'month'
and not blacklisted_mcc
and trim(tnc_country_group) = 'DEU'
union all
select 
'users with limit btw 40€ and 500€' as label, 
txn_date,
n_40_500_users
from deu_elig_card_txns_df ect
where timeframe = 'month'
and not blacklisted_mcc
and trim(tnc_country_group) = 'DEU'
union all
select 
'users with limit btw 30€ and 500€' as label, 
txn_date,
n_30_500_users
from deu_elig_card_txns_df ect
where timeframe = 'month'
and not blacklisted_mcc
and trim(tnc_country_group) = 'DEU'
union all
select 
'users with limit btw 20€ and 500€' as label, 
txn_date,
n_20_500_users
from deu_elig_card_txns_df ect
where timeframe = 'month'
and not blacklisted_mcc
and trim(tnc_country_group) = 'DEU'
union all
select 
'users with limit btw 10€ and 500€' as label, 
txn_date,
n_10_500_users
from deu_elig_card_txns_df ect
where timeframe = 'month'
and not blacklisted_mcc
and trim(tnc_country_group) = 'DEU'
)
select
label,
round(avg(n_users)::numeric/1000, 1) as avg_users_k,
round(avg(n_users)::numeric/min(geu.n_eligible_maus), 3)*100  as perc_elig_maus
from unions
inner join elig_users_df geu
on 1=1 
where trim(tnc_country_group) = 'DEU'
group by 1
order by 2
"""
lower_limit_df = con.execute(lower_limit_query).fetchdf()
lower_limit_df

,label,avg_users_k,perc_elig_maus
0,eligible users,217.2,40.0
1,users with limit btw 40€ and 500€,239.3,44.1
2,users with limit btw 30€ and 500€,266.0,49.0
3,users with limit btw 20€ and 500€,294.7,54.3
4,users with limit btw 10€ and 500€,331.7,61.1


In [19]:
af.column_single_label(
    lower_limit_df, af.teal, "label", "perc_elig_maus", 800, 400, "y"
)

alt.LayerChart(...)

<a id='section2.2'></a>
## Increasing Upper Limits

On the other hand, increasing the upper transaction limit from 500€ until up to 1000€ seems to have almost no effect in increasing the number of users with eligible transactions, we only see a 0.7pp increase when increasing the upper limit all the way to 1000€.

In [20]:
upper_limit_query = """
with unions as (
select 
'eligible users' as label, 
txn_date,
n_txn_eligible_users as n_users
from deu_elig_card_txns_df ect
where timeframe = 'month'
and not blacklisted_mcc
and trim(tnc_country_group) = 'DEU'
union all
select 
'users with limit btw 50€ and 600€' as label, 
txn_date,
n_50_600_users
from deu_elig_card_txns_df ect
where timeframe = 'month'
and not blacklisted_mcc
and trim(tnc_country_group) = 'DEU'
union all
select 
'users with limit btw 50€ and 700€' as label, 
txn_date,
n_50_700_users
from deu_elig_card_txns_df ect
where timeframe = 'month'
and not blacklisted_mcc
and trim(tnc_country_group) = 'DEU'
union all
select 
'users with limit btw 50€ and 800€' as label, 
txn_date,
n_50_800_users
from deu_elig_card_txns_df ect
where timeframe = 'month'
and not blacklisted_mcc
and trim(tnc_country_group) = 'DEU'
union all
select 
'users with limit btw 50€ and 900€' as label, 
txn_date,
n_50_900_users
from deu_elig_card_txns_df ect
where timeframe = 'month'
and not blacklisted_mcc
and trim(tnc_country_group) = 'DEU'
union all
select 
'users with limit btw 50€ and 1000€' as label, 
txn_date,
n_50_1000_users
from deu_elig_card_txns_df ect
where timeframe = 'month'
and not blacklisted_mcc
and trim(tnc_country_group) = 'DEU'
)
select
label,
round(avg(n_users)::numeric/1000, 1) as avg_users_k,
round(avg(n_users)::numeric/min(geu.n_eligible_maus), 3)*100  as perc_elig_maus
from unions
inner join elig_users_df geu
on 1=1
where trim(tnc_country_group) = 'DEU'
group by 1
order by 2
"""
upper_limit_df = con.execute(upper_limit_query).fetchdf()
upper_limit_df

,label,avg_users_k,perc_elig_maus
0,eligible users,217.2,40.0
1,users with limit btw 50€ and 600€,218.4,40.3
2,users with limit btw 50€ and 700€,219.3,40.4
3,users with limit btw 50€ and 800€,220.0,40.5
4,users with limit btw 50€ and 900€,220.5,40.6
5,users with limit btw 50€ and 1000€,220.8,40.7


In [21]:
af.column_single_label(
    upper_limit_df, af.teal, "label", "perc_elig_maus", 800, 400, "y"
)

alt.LayerChart(...)

<a id='section2.3'></a>
## Reconsidering Blacklisted MCCs

Not all card transactions are eligible for this product, we exclude some merchant categories (MCCs) to reduce risk (you can find a comprehensive list of excluded MCCs [here](https://docs.google.com/spreadsheets/d/1Bn-0BHBPztNrjDj1yudYwDDjJFzLnmDXAbmEMApMFJ4/edit#gid=975436947).  In order to see how many users we could add to increase the number of users with eligible transactions, we start by excluding users who already have an eligible transaction, and see how many we have remaining. For example, we can see that if we were to not exclude ATM transactions from the product moving forward, we would add about 140.7K users to the group of users with eligible transactions (25.9% of all eligible MAUs). 

On the other hand, we can see that 5% of users who already have eligible transactions also have transactions with blocked MCCs.

In [22]:
blacklisted_mccs_query = """
select 
mcc,
coalesce(mcc_description, 'Eligible users with blacklisted MCC transactions') as mcc_description, 
mcc_category,
avg(n_txn_eligible_users)::int as avg_users,
round(avg(n_txn_eligible_users)::numeric/min(geu.n_eligible_maus), 3)*100  as perc_elig_maus
from deu_elig_card_txns_df ect
inner join elig_users_df geu
on timeframe = 'month'
and blacklisted_mcc
and ect.tnc_country_group = geu.tnc_country_group
and trim(ect.tnc_country_group) = 'DEU'
group by 1, 2, 3
order by 4 desc 
limit 10
"""
blacklisted_mccs_df = con.execute(blacklisted_mccs_query).fetchdf()
blacklisted_mccs_df

,mcc,mcc_description,mcc_category,avg_users,perc_elig_maus
0,6011.0,Financial institutions — automated cash disbur...,atm,140703,25.9
1,5411.0,Groceries and supermarkets,grocery_market,85152,15.7
2,5812.0,Eating places and restaurants,restaurants,53641,9.9
3,5541.0,Service stations (with or without ancillary se...,gas_service_station,47619,8.8
4,NaN,Eligible users with blacklisted MCC transactions,NaN,27138,5.0
5,5912.0,Drug stores and pharmacies,drug_pharma,17082,3.1
6,8999.0,Professional services — not elsewhere classified,prof_serv,12436,2.3
7,4722.0,Travel agencies and tour operators,travel_tour_agencies,12403,2.3
8,4829.0,Wire transfers and money orders,money_cash_financial,11733,2.2
9,5499.0,Miscellaneous food shops — convenience and spe...,food_drink_stores,10038,1.9


<a id='section3'></a>
# What does eligibility look like for other core markets outside of Germany?

<a id='section3.1'></a>
## All Eligible Users - All Legal Entities

When looking into user eligibility for the Installments Loans product, we can see that France is the market with the most eligible users (389K users), which corresponds to about 60% of the number of eligible users we saw for Germany.

In [23]:
grouped_elig_users_query = """
select 
tnc_country_group,
'All KYCc users (K)' as label,
kycc_users::numeric/1000 as value 
from elig_users_df
where trim(tnc_country_group) != 'DEU'
union all 
select 
tnc_country_group,
'All MAUs (K)' as label,
n_maus::numeric/1000 as value 
from elig_users_df
where trim(tnc_country_group) != 'DEU'
union all 
select 
tnc_country_group,
'Lisbon Eligible Users (K)' as label,
n_lisbon_eligible_users::numeric/1000 as value 
from elig_users_df
where trim(tnc_country_group) != 'DEU'
union all 
select 
tnc_country_group,
'Installments Eligible Users (K)' as label,
n_eligible_users::numeric/1000 as value 
from elig_users_df
where trim(tnc_country_group) != 'DEU'
union all 
select 
tnc_country_group,
'Installments Eligible MAUs (K)' as label,
n_eligible_maus::numeric/1000 as value 
from elig_users_df
where trim(tnc_country_group) != 'DEU'
"""
con.register("elig_users_df", elig_users_df)
grouped_elig_users_df = con.execute(grouped_elig_users_query).fetchdf()

In [24]:
af.column_multi(
    grouped_elig_users_df,
    "tnc_country_group",
    "label",
    "value",
    "tnc_country_group",
    180,
    400,
    "-y",
)

alt.Chart(...)

<a id='section3.2'></a>
## All Eligible Users - EU Legal Entity

For the first iteration of the launch of Installment Loans in markets other than Germany, we are only considering users with German IBANs (EU legal entity). Since France and Austria don't have local IBANs, only Italy and Spain are impacted when we filter for users with an EU legal entity. Looking at all KYCc users, we see a reduction of 304K users in Spain, and 297K users in Italy.

In [25]:
grouped_elig_eu_users_query = """
select 
tnc_country_group,
'All EU Legal KYCc users (K)' as label,
kycc_users_eu_legal::numeric/1000 as value 
from elig_users_df
where trim(tnc_country_group) != 'DEU'
union all 
select 
tnc_country_group,
'All EU Legal MAUs (K)' as label,
n_maus_eu_legal::numeric/1000 as value 
from elig_users_df
where trim(tnc_country_group) != 'DEU'
union all 
select 
tnc_country_group,
'EU Legal Lisbon Eligible Users (K)' as label,
n_lisbon_eligible_users_eu_legal::numeric/1000 as value 
from elig_users_df
where trim(tnc_country_group) != 'DEU'
union all 
select 
tnc_country_group,
'EU Legal Installments Eligible Users (K)' as label,
n_eligible_users_eu_legal::numeric/1000 as value 
from elig_users_df
where trim(tnc_country_group) != 'DEU'
union all 
select 
tnc_country_group,
'EU Legal Installments Eligible MAUs (K)' as label,
n_eligible_maus_eu_legal::numeric/1000 as value 
from elig_users_df
where trim(tnc_country_group) != 'DEU'
"""
con.register("elig_users_df", elig_users_df)
grouped_elig_eu_users_df = con.execute(grouped_elig_eu_users_query).fetchdf()

In [26]:
af.column_multi(
    grouped_elig_eu_users_df,
    "tnc_country_group",
    "label",
    "value",
    "tnc_country_group",
    180,
    400,
    "-y",
)

alt.Chart(...)

<a id='section3.3'></a>
## Percentage of eligible users - All Legal Entities

When comparing the 2 percentages for other markets, we can see substantially bigger differences between these than what we saw for Germany. This might be due to the fact that we also have bigger gaps between the number of KYCc users and the number of MAUs in these markets.

In [27]:
grouped_elig_users_perc_query = """
select 
tnc_country_group,
'% eligible users out of kycc users' as label,
perc_eligible_users_out_of_kycc_users as value 
from elig_users_df
where trim(tnc_country_group) != 'DEU'
union all 
select 
tnc_country_group,
'% eligible MAUs out of MAUs' as label,
perc_eligible_maus_out_of_maus as value 
from elig_users_df
where trim(tnc_country_group) != 'DEU'
"""
grouped_elig_users_perc_df = con.execute(grouped_elig_users_perc_query).fetchdf()

In [28]:
af.bar_multi(
    grouped_elig_users_perc_df,
    "tnc_country_group:N",
    "value",
    "label",
    "tnc_country_group",
    600,
    120,
    "-y",
)

alt.Chart(...)

<a id='section3.4'></a>
## Percentage of eligible users - EU Legal Entities

Once again, only Italy and Spain show changes when filtering for EU Legal entities only. We seem to have a polarizing effect here, with the % eligible users out of KYCc users decreasing for both countries, and the % eligible MAUs out of MAUs increasing to 77.7% in Spain and 78.5% in Italy.

In [29]:
grouped_elig_users_perc_eu_query = """
select 
tnc_country_group,
'% eligible users out of kycc users (EU Legal Entity)' as label,
perc_eligible_users_out_of_kycc_users_eu_legal as value 
from elig_users_df
where trim(tnc_country_group) != 'DEU'
union all 
select 
tnc_country_group,
'% eligible MAUs out of MAUs (EU Legal Entity)' as label,
perc_eligible_maus_out_of_maus_eu_legal as value 
from elig_users_df
where trim(tnc_country_group) != 'DEU'
"""
grouped_elig_users_perc_eu_df = con.execute(grouped_elig_users_perc_eu_query).fetchdf()

In [30]:
af.bar_multi(
    grouped_elig_users_perc_eu_df,
    "tnc_country_group:N",
    "value",
    "label",
    "tnc_country_group",
    600,
    120,
    "-y",
)

alt.Chart(...)

<a id='section3.5'></a>
## Card Transactions Eligibility

For the exercise below, we will once again be looking into what percentage of eligible MAUs have eligible transactions in each market (with no legal entity filters). The main difference between these markets and Germany is that here we have lower upper limits - since we only consider eligible transactions between 50€ and 200€, we've updated these limits accordingly. We can see a very similar pattern in all markets as we also saw for Germany: a steep increase of eligibility when lowering the limits (reaching about 68.8% of eligible users in France and Italy), and small increases in eligibility when increasing the upper limits.

In [31]:
lower_limit_query = """
with unions as (
select 
tnc_country_group,
'eligible users' as label, 
txn_date,
n_txn_eligible_users as n_users
from oth_elig_card_txns_df ect
where timeframe = 'month'
and not blacklisted_mcc
and trim(tnc_country_group) != 'DEU'
union all
select 
tnc_country_group,
'users with limit btw 40€ and 200€' as label, 
txn_date,
n_40_200_users
from oth_elig_card_txns_df ect
where timeframe = 'month'
and not blacklisted_mcc
and trim(tnc_country_group) != 'DEU'
union all
select 
tnc_country_group,
'users with limit btw 30€ and 200€' as label, 
txn_date,
n_30_200_users
from oth_elig_card_txns_df ect
where timeframe = 'month'
and not blacklisted_mcc
and trim(tnc_country_group) != 'DEU'
union all
select 
tnc_country_group,
'users with limit btw 20€ and 200€' as label, 
txn_date,
n_20_200_users
from oth_elig_card_txns_df ect
where timeframe = 'month'
and not blacklisted_mcc
and trim(tnc_country_group) != 'DEU'
union all
select 
tnc_country_group,
'users with limit btw 10€ and 200€' as label, 
txn_date,
n_10_200_users
from oth_elig_card_txns_df ect
where timeframe = 'month'
and not blacklisted_mcc
and trim(tnc_country_group) != 'DEU'
)
select
u.tnc_country_group,
label,
round(avg(n_users)::numeric/1000, 1) as avg_users_k,
round(avg(n_users)::numeric/min(geu.n_eligible_maus), 3)*100  as perc_elig_maus
from unions u
inner join elig_users_df geu
using(tnc_country_group)
where trim(u.tnc_country_group) != 'DEU'
group by 1,2
order by 1,3
"""
lower_limit_df = con.execute(lower_limit_query).fetchdf()

In [32]:
af.column_multi(
    lower_limit_df,
    "tnc_country_group",
    "label",
    "perc_elig_maus",
    "tnc_country_group",
    180,
    400,
    "y",
)

alt.Chart(...)

In [33]:
upper_limit_query = """
with unions as (
select 
tnc_country_group,
'eligible users' as label, 
txn_date,
n_txn_eligible_users as n_users
from oth_elig_card_txns_df ect
where timeframe = 'month'
and not blacklisted_mcc
and trim(tnc_country_group) != 'DEU'
union all
select 
tnc_country_group,
'users with limit btw 50€ and 300€' as label, 
txn_date,
n_50_300_users
from oth_elig_card_txns_df ect
where timeframe = 'month'
and not blacklisted_mcc
and trim(tnc_country_group) != 'DEU'
union all
select 
tnc_country_group,
'users with limit btw 50€ and 400€' as label, 
txn_date,
n_50_400_users
from oth_elig_card_txns_df ect
where timeframe = 'month'
and not blacklisted_mcc
and trim(tnc_country_group) != 'DEU'
union all
select 
tnc_country_group,
'users with limit btw 50€ and 500€' as label, 
txn_date,
n_50_500_users
from oth_elig_card_txns_df ect
where timeframe = 'month'
and not blacklisted_mcc
and trim(tnc_country_group) != 'DEU'
)
select
tnc_country_group,
label,
round(avg(n_users)::numeric/1000, 1) as avg_users_k,
round(avg(n_users)::numeric/min(geu.n_eligible_maus), 3)*100  as perc_elig_maus
from unions
inner join elig_users_df geu
using(tnc_country_group)
where trim(tnc_country_group) != 'DEU'
group by 1, 2
order by 3
"""
upper_limit_df = con.execute(upper_limit_query).fetchdf()

In [34]:
af.column_multi(
    upper_limit_df,
    "tnc_country_group",
    "label",
    "perc_elig_maus",
    "tnc_country_group",
    180,
    400,
    "y",
)

alt.Chart(...)

<a id='section4'></a>
# Can we increase eligibility by adding other transaction types?

We also explored how many users we could add to our transaction eligibility pool if we were to include transactions beyond card transactions, namely Direct Debits (DDs) and Direct Transfers (DTs). Germany is the best performing market in this case - even though more than half of eligible MAUs have each of these transaction types, these additions seem to be less impactful than adding some of the top MCCs above, we only have 12% of users with DTs that don't already have eligible transactions (and 9.6% for DDs). 

For all other markets, the percentage of users we could add with these transaction types would be even lower (once again, we are considering eligible transactions between 50€ and 500€ for Germany, and between 50€ and 200€ for other markets).

In [35]:
month_elig_all_txns_query = """
select 
tnc_country_group,
feature,
avg(ect.n_users)::int/ 1000 as avg_txn_k_users,
round(avg(ect.n_users)::numeric/min(geu.n_eligible_maus), 3)*100 as perc_elig_maus,
avg(ect.n_users_excl_elig_txns)::int/ 1000 as avg_txn_k_users_excl_elig_txns,
round(avg(ect.n_users_excl_elig_txns)::numeric/min(geu.n_eligible_maus), 3)*100 as perc_elig_maus_excl_elig_txns
from all_elig_txns_df ect
inner join elig_users_df geu
using (tnc_country_group)
group by 1, 2
order by 1, 2
"""
con.register("all_elig_txns_df", all_elig_txns_df)
month_elig_all_txns_df = con.execute(month_elig_all_txns_query).fetchdf()
month_elig_all_txns_df

,tnc_country_group,feature,avg_txn_k_users,perc_elig_maus,avg_txn_k_users_excl_elig_txns,perc_elig_maus_excl_elig_txns
0,AUT,n_dd,14,28.1,2,5.4
1,AUT,n_dt,23,44.5,5,9.5
2,DEU,n_dd,275,50.7,52,9.6
3,DEU,n_dt,317,58.4,65,12.0
4,ESP,n_dd,24,22.6,6,5.7
5,ESP,n_dt,40,36.8,10,9.1
6,FRA,n_dd,52,17.7,8,2.8
7,FRA,n_dt,101,34.6,20,6.8
8,ITA,n_dd,21,9.8,3,1.5
9,ITA,n_dt,67,30.5,11,5.2


In [36]:
af.column_multi(
    month_elig_all_txns_df,
    "tnc_country_group",
    "feature",
    "perc_elig_maus_excl_elig_txns",
    "tnc_country_group",
    100,
    400,
    "y",
)

alt.Chart(...)